# Rapport TP2

<!-- TODO: ajouter les noms et matricules -->
Xavier Jouslin - 2073104

**Note: to be able to run the code**
```bash
pip install scikit-image
```

## Question 1

### Description generale de l'algorithme de Karhunen-Loève

Soit une image compose de plusieurs base(e.g RGB ou YUV), ces representation sont conforme et norme mais pas forcemment le plus optimiser, l'algorithme de Karhunen-Loève permet de trouver la base avec le plus d'information possible avec le moins de base possible. Autrement dit, il faut trouver l'entropie maximale pour chaque composantes choisie de tel maniere que le nombre de composantes reste minimal.

### Etape de l'algorithme de Karhunen-Loève

Pour compresser avec la transforme de Karhunen-Loève, il faut determiner la moyenne de la base de l'image puis la covariance de l'image, puis trouver le vecteur propre et enfin trouver les composantes de la nouvelle base.

### Difference avec les autres methodes de transformation

#### Discrete Cosine Transform

La transforme en cosinus discret utilise le principe de frequence de cosinus. Cette transformation est moins optimal, plus rapide que l'algorithme de Karhunen-Loève. L'avantage avec cette algorithme est qu'il est parrallelisable.

#### Odelette

l'algorithme de Odelette utilise le principe des ondes a la difference des cosinus avec des frequence differentes pour emcode les images. Cette algorithme est plus rapide mais moins optimal que la transforme de Karhunen-Loève. 

## Question 2

### Description de la solution

ImageKL est l'implementation de la compression d'image avec l'algorithme de Karhunen-Loeve. Chaque image est transforme en matrice 2D(taille de l'image * canale colorimetrique). La methode encodage permet de compresser l'image avec ou non la possibilite de change de base colorimetrique.

In [95]:
from os import path
import numpy as np
import cv2
import matplotlib.pyplot as plt
from einops import rearrange

class ImageKL:
    def __init__(self, path_code: str):
        self.__width = 0
        self.__height = 0
        self.__quantifier = ()
        self.__compress_code = None
        self.code = self.tilde_image(path_code)

    def tilde_image(self, image_path: str):
        image = plt.imread(image_path)
        self.__width = image.shape[1]
        self.__height = image.shape[0]
        return rearrange(image, 'h w c -> (h w) c')

    def psnr(self):
        return 10 * np.log10(self.__mse()) + 20 * np.log10(self.__msi())

    def ssim(self):
        covariance_images = np.cov(np.concatenate((np.reshape(self.code, (-1, 1)), np.reshape(self.__compress_code, (-1, 1))), axis=1).T)[0,1]
        variance_base = np.cov(np.reshape(self.code, (-1, 1)).T)
        variance_compressed = np.cov(np.reshape(self.__compress_code, (-1, 1)).T)
        return (2 * np.mean(self.code) * np.mean(self.__compress_code) + 0.01) * (2 * covariance_images + 0.03) / ((np.mean(self.code) ** 2 + np.mean(self.__compress_code) ** 2 + 0.01) * (variance_base + variance_compressed + 0.03))

    def compression_ratio(self):
        code_color_space = np.power(2, 8 * 3)
        compress_code_color_space = np.power(2, self.__quantifier[0]) * np.power(2, self.__quantifier[1]) * np.power(2, self.__quantifier[2])
        code_length = np.array(self.code).shape[0] * np.array(self.code).shape[1]
        compress_code = np.array(self.__compress_code).shape[0] * np.array(self.__compress_code).shape[1]
        return compress_code * compress_code_color_space / (code_length * code_color_space)

    def rgb2yuv(self, rgb):
        return rearrange(cv2.cvtColor(rearrange(rgb, '(h w) c -> h w c', h=self.__height, w=self.__width).astype(np.float32), cv2.COLOR_RGB2YUV), 'h w c -> (h w) c')

    def yuv2rgb(self, yuv):
        return rearrange(cv2.cvtColor(rearrange(yuv, '(h w) c -> h w c', h=self.__height, w=self.__width).astype(np.float32), cv2.COLOR_YUV2RGB), 'h w c -> (h w) c')
        

    def encode(self, quantifier=(8, 8, 8), is_yuv=False):
        if is_yuv:
            self.code = self.rgb2yuv(self.code)
        kl_transform = self.kl_transform()
        self.__quantifier = quantifier
        self.__compress_code = self.kl_transform_inv(self.reconstruct(self.quantify(quantifier, kl_transform), kl_transform))
        if is_yuv:
            self.__compress_code = self.yuv2rgb(self.__compress_code)
        return rearrange(np.clip(self.__compress_code, 0, 1), '(h w) c -> h w c', h=self.__height, w=self.__width)

    def __average(self, code):
        mean = np.mean(code, axis=0)
        average = np.zeros(code.shape)
        for i in range(code.shape[1]):
            average[:, i] = mean[i]
        return average

    def __covariant(self, code):
        return np.cov(code, rowvar=False)

    def __eigenvector(self, code):
        return np.linalg.eig(self.__covariant(code))[1]

    def __inverse_eigenvector(self, code):
        return np.linalg.inv(self.__eigenvector(code))

    def kl_transform(self):
        return np.transpose(np.matmul(self.__eigenvector(self.code), np.transpose(np.subtract(self.code, self.__average(self.code)))))

    def quantify(self, quantifier, kl_transform_outpout):
        canals = [kl_transform_outpout[:, i] for i in range(3)]
        return  [np.linspace(np.min(canals[i]), np.max(canals[i]), num=2**quantifier[i]) for i in range(len(canals))]

    def reconstruct(self, quantifier, kl_transform_outpout):
        canals = [kl_transform_outpout[:, i] for i in range(3)]
        compress_canals = [[quantifier[i][np.abs(quantifier[i] - data).argmin()] for data in canals[i]] for i in range(len(quantifier))]
        return np.transpose(np.array(compress_canals))

    def kl_transform_inv(self, kl_transform):
        return np.transpose(np.matmul(np.transpose(self.__inverse_eigenvector(self.code)), np.transpose(np.add(kl_transform, self.__average(self.code)))))

    def __msi(self):
        return max(np.power(2, self.__quantifier))

    def __mse(self):
        return 1 / np.mean((self.__compress_code - self.code) ** 2)


In [96]:
import pandas as pd
import os

notebook_path = os.path.abspath("rapport.ipynb")
script_path = os.path.dirname(notebook_path)
image_files = [os.path.join(script_path, "../data", f) for f in os.listdir(os.path.join(script_path, "../data")) if os.path.isfile(os.path.join(script_path, "../data", f))]
quantifiers = [(8, 8, 8), (8, 8, 4), (8, 8, 0), (8, 4, 0)]
result = pd.DataFrame(columns=["image", "canal", "is_yuv", "psnr", "ssim", "compression_ratio"])
for image_file in image_files:
    image = ImageKL(image_file)
    for quantifier in quantifiers:
        for is_yuv in [True, False]:
            image.encode(quantifier, is_yuv)
            image_result = {"image": [path.basename(image_file)], "canal": [", ".join([str(canal) for canal in quantifier])], "is_yuv": [is_yuv], "psnr": [image.psnr()], "ssim": [image.ssim()], "compression_ratio": [image.compression_ratio()]}
            image_result = pd.DataFrame(image_result)
            result = pd.concat([result, image_result], ignore_index=True)
print(result)

          image    canal is_yuv       psnr      ssim  compression_ratio
0   kodim13.png  8, 8, 8   True  51.963981  0.032412           1.000000
1   kodim13.png  8, 8, 8  False  59.702867  0.519967           1.000000
2   kodim13.png  8, 8, 4   True  49.459605 -0.070631           0.062500
3   kodim13.png  8, 8, 4  False  53.202861  0.148161           0.062500
4   kodim13.png  8, 8, 0   True  42.271890 -0.035542           0.003906
5   kodim13.png  8, 8, 0  False  49.127213 -0.390556           0.003906
6   kodim13.png  8, 4, 0   True  46.539751 -0.017038           0.000244
7   kodim13.png  8, 4, 0  False  53.483336  0.115275           0.000244
8   kodim23.png  8, 8, 8   True  44.725769  0.039733           1.000000
9   kodim23.png  8, 8, 8  False  52.227112  0.051631           1.000000
10  kodim23.png  8, 8, 4   True  50.249700  0.062748           0.062500
11  kodim23.png  8, 8, 4  False  53.306601  0.153631           0.062500
12  kodim23.png  8, 8, 0   True  48.376208 -0.020716           0

## Question 3

Nous allons trie le tableau que nous avons obtenu dans la section precedente de telle facon d'optenir le meilleure modele ayant le meilleure resultat.

In [97]:
best_result = result.sort_values(by=["image", "psnr", "ssim", "compression_ratio"], ascending=[True, False, False, True])
best_result = best_result.groupby("image").head(1)
print(best_result)

          image    canal is_yuv       psnr      ssim  compression_ratio
37  kodim01.png  8, 8, 0  False  60.995416  0.438610           0.003906
17  kodim02.png  8, 8, 8  False  68.281711  0.892883           1.000000
31  kodim05.png  8, 4, 0  False  55.523333  0.229688           0.000244
1   kodim13.png  8, 8, 8  False  59.702867  0.519967           1.000000
15  kodim23.png  8, 4, 0  False  55.714587  0.217189           0.000244


On observe que les meilleures resultat se trouve avec des quantificateurs(canal) possedant une tres forte etalement dans l espace du rouge et du vert. Dans les echantillons pris dans les donnees, il y a preponderance de rouge et de vert. On observe aussi que la base colorimetrique dans nos meilleures resultats correspond au RGB plutot que YUV. YUV devrait permettre une meilleure compression. la matrice KL de l'image YUV permet de differencie la luminosite de l'image versus l'information de la couleur de l'image.

## Question 4

In [98]:
class ImageKLFeat:
    def __init__(self, path_code_fit_image: str, path_code_test_image: str):
        self.__compress_code = None
        self.__quantifier = ()
        self.code = ImageKL(path_code_fit_image)
        self.test_code = ImageKL(path_code_test_image)

    def psnr(self):
        return 10 * np.log10(self.__mse()) + 20 * np.log10(self.__msi())

    def ssim(self):
        covariance_images = np.cov(np.concatenate((np.reshape(self.test_code.code, (-1, 1)), np.reshape(self.__compress_code, (-1, 1))), axis=1).T)[0,1]
        variance_base = np.cov(np.reshape(self.test_code.code, (-1, 1)).T)
        variance_compressed = np.cov(np.reshape(self.__compress_code, (-1, 1)).T)
        return (2 * np.mean(self.test_code.code) * np.mean(self.__compress_code) + 0.01) * (2 * covariance_images + 0.03) / ((np.mean(self.test_code.code) ** 2 + np.mean(self.__compress_code) ** 2 + 0.01) * (variance_base + variance_compressed + 0.03))

    def encode(self, quantifier=(8, 8, 8), is_yuv=False):
        self.__quantifier = quantifier
        if is_yuv:
            self.code = self.rgb2yuv(self.code)
            self.test_code = self.rgb2yuv(self.test_code)
        kl_transform = self.code.kl_transform()
        self.__compress_code = self.test_code.kl_transform_inv(self.test_code.reconstruct(self.code.quantify(quantifier, kl_transform), kl_transform))
        return self.__compress_code
    
    def __msi(self):
        return max(np.power(2, self.__quantifier))

    def __mse(self):
        return 1 / np.mean((self.__compress_code - self.test_code.code) ** 2)


In [99]:
compute_image_files = [[(f, f1) for f1 in image_files if f1 != f] for f in image_files]
result = pd.DataFrame(columns=["image", "canal", "is_yuv", "psnr", "ssim"])
for image_file in compute_image_files:
    for combination in image_file:        
        image = ImageKLFeat(combination[0], combination[1])
        best_colorimetric_base = best_result.loc[best_result["image"] == path.basename(combination[0])]["is_yuv"].values[0]
        best_quantifier = [int(quant) for quant in best_result.loc[best_result["image"] == path.basename(combination[1])]["canal"].values[0].split(", ")]
        image.encode(quantifier, best_colorimetric_base)
        image_result = {"image": ", ".join([path.basename(combination[0]), path.basename(combination[1])]), "canal": [", ".join([str(canal) for canal in quantifier])], "is_yuv": [is_yuv], "psnr": [image.psnr()], "ssim": [image.ssim()]}
        image_result = pd.DataFrame(image_result)
        result = pd.concat([result, image_result], ignore_index=True)
print(result)

                       image    canal is_yuv       psnr      ssim
0   kodim13.png, kodim23.png  8, 4, 0  False  49.848191 -0.013938
1   kodim13.png, kodim02.png  8, 4, 0  False  56.529369  0.420328
2   kodim13.png, kodim05.png  8, 4, 0  False  51.515010 -0.065829
3   kodim13.png, kodim01.png  8, 4, 0  False  56.783313  0.332908
4   kodim23.png, kodim13.png  8, 4, 0  False  50.654008 -0.199759
5   kodim23.png, kodim02.png  8, 4, 0  False  59.428893  0.567676
6   kodim23.png, kodim05.png  8, 4, 0  False  52.138025 -0.100200
7   kodim23.png, kodim01.png  8, 4, 0  False  57.912715  0.335194
8   kodim02.png, kodim13.png  8, 4, 0  False  50.786554 -0.176172
9   kodim02.png, kodim23.png  8, 4, 0  False  50.295179  0.064563
10  kodim02.png, kodim05.png  8, 4, 0  False  52.579081 -0.102978
11  kodim02.png, kodim01.png  8, 4, 0  False  59.292190  0.397960
12  kodim05.png, kodim13.png  8, 4, 0  False  50.638600 -0.203653
13  kodim05.png, kodim23.png  8, 4, 0  False  50.115935 -0.008152
14  kodim0

## Question 5

On observe selon nos resultats que la qualite de la compression est deteriore. Il est plus interressant d'utilise la meme image pour obtenir la matrice KL et la matrice de quantification.